In [1]:
import os
import time
import io
import hashlib
import numpy as np
from PIL import Image
from utils import parseBBOXFile, getTagValue

In [2]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import tensorflow as tf
from object_detection.utils import dataset_util, label_map_util

In [4]:
label_map = label_map_util.load_labelmap('../annotations/label_map.pbtxt')
label_map_dict = label_map_util.get_label_map_dict(label_map)
label_map_dict

{'car': 1, 'truck': 2, 'bus': 3, 'bike': 4, 'person': 5}

Convert a folder of images + xml files to TF-record format, needed for training.

In [6]:
def getFiles(path):
    files = []
    for file in os.listdir(path):
        with open('{}/{}'.format(path, file), 'r') as f:
            files += [item.strip().lower() for item in f]
    return files

In [7]:
def convertFolder(name, image_path, annotate_path, image_format = b'jpg'):
    files = getFiles(image_path)
    if len(files) == 0:
        return
    
    print("Creating TF-record:", '{}/{}.record'.format(annotate_path, name))
    with tf.io.TFRecordWriter('{}/{}.record'.format(annotate_path, name)) as writer:
        for f in files:
            img, xml = '../images/{}.jpg'.format(f), '../images/{}.xml'.format(f)
            encoded_jpg = None
            with tf.io.gfile.GFile(img, 'rb') as fid:
                encoded_jpg = fid.read()

            image = Image.open(io.BytesIO(encoded_jpg))
            key = hashlib.sha256(encoded_jpg).hexdigest().encode('utf8')
            width, height = image.size
            filename = '{}.jpg'.format(f.split('/')[-1]).encode('utf8')
            xmins, xmaxs = [], []
            ymins, ymaxs = [], []
            classes_name, classes_id = [], []
            truncated = []
            poses = []
            difficult = []

            for obj in parseBBOXFile(xml):
                lbl = getTagValue(obj, 'name')

                xmin = (float(getTagValue(obj, 'xmin')) / float(width))
                xmax = (float(getTagValue(obj, 'xmax')) / float(width))

                ymin = (float(getTagValue(obj, 'ymin')) / float(height))
                ymax = (float(getTagValue(obj, 'ymax')) / float(height))

                classes_name.append(lbl.encode('utf8'))
                classes_id.append(label_map_dict[lbl])
                xmins.append(xmin)
                xmaxs.append(xmax)
                ymins.append(ymin)
                ymaxs.append(ymax)
                
                truncated.append(int(getTagValue(obj, 'truncated')))
                difficult.append(int(getTagValue(obj, 'difficult')))
                poses.append(getTagValue(obj, 'pose').encode('utf8'))

            record = tf.train.Example(features=tf.train.Features(feature={
                'image/height': dataset_util.int64_feature(height),
                'image/width': dataset_util.int64_feature(width),
                'image/filename': dataset_util.bytes_feature(filename),
                'image/source_id': dataset_util.bytes_feature(filename),
                'image/key/sha256': dataset_util.bytes_feature(key),
                'image/encoded': dataset_util.bytes_feature(encoded_jpg),
                'image/format': dataset_util.bytes_feature(image_format),
                'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
                'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
                'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
                'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
                'image/object/class/text': dataset_util.bytes_list_feature(classes_name),
                'image/object/class/label': dataset_util.int64_list_feature(classes_id),
                'image/object/difficult': dataset_util.int64_list_feature(difficult),
                'image/object/truncated': dataset_util.int64_list_feature(truncated),
                'image/object/view': dataset_util.bytes_list_feature(poses),
            })).SerializeToString()
            writer.write(record)
            print("Wrote", f, 'to the record.')
            #break
    print("Finished!")

In [8]:
convertFolder('train', '../images/train', '../annotations')

Creating TF-record: ../annotations/train.record
Wrote dark/2015_01879 to the record.
Wrote dark/2015_01880 to the record.
Wrote dark/2015_01881 to the record.
Wrote dark/2015_01882 to the record.
Wrote dark/2015_01883 to the record.
Wrote dark/2015_01884 to the record.
Wrote dark/2015_01885 to the record.
Wrote dark/2015_01886 to the record.
Wrote dark/2015_01889 to the record.
Wrote dark/2015_01890 to the record.
Wrote dark/2015_01891 to the record.
Wrote dark/2015_01892 to the record.
Wrote dark/2015_01893 to the record.
Wrote dark/2015_01894 to the record.
Wrote dark/2015_01896 to the record.
Wrote dark/2015_01897 to the record.
Wrote dark/2015_01898 to the record.
Wrote dark/2015_01899 to the record.
Wrote dark/2015_01900 to the record.
Wrote dark/2015_01903 to the record.
Wrote dark/2015_01905 to the record.
Wrote dark/2015_01906 to the record.
Wrote dark/2015_01907 to the record.
Wrote dark/2015_01908 to the record.
Wrote dark/2015_01909 to the record.
Wrote dark/2015_01910 to th

Wrote dark/2015_02131 to the record.
Wrote dark/2015_02132 to the record.
Wrote dark/2015_02133 to the record.
Wrote dark/2015_02134 to the record.
Wrote dark/2015_02135 to the record.
Wrote dark/2015_02136 to the record.
Wrote dark/2015_02137 to the record.
Wrote dark/2015_02140 to the record.
Wrote dark/2015_02141 to the record.
Wrote dark/2015_02142 to the record.
Wrote dark/2015_02143 to the record.
Wrote dark/2015_02145 to the record.
Wrote dark/2015_02146 to the record.
Wrote dark/2015_02147 to the record.
Wrote dark/2015_02148 to the record.
Wrote dark/2015_02149 to the record.
Wrote dark/2015_02150 to the record.
Wrote dark/2015_02151 to the record.
Wrote dark/2015_02153 to the record.
Wrote dark/2015_02154 to the record.
Wrote dark/2015_02155 to the record.
Wrote dark/2015_02156 to the record.
Wrote dark/2015_02157 to the record.
Wrote dark/2015_02158 to the record.
Wrote dark/2015_02159 to the record.
Wrote dark/2015_02160 to the record.
Wrote dark/2015_02161 to the record.
W

Wrote dark/2015_02376 to the record.
Wrote dark/2015_02377 to the record.
Wrote dark/2015_02378 to the record.
Wrote dark/2015_02379 to the record.
Wrote dark/2015_02380 to the record.
Wrote dark/2015_02381 to the record.
Wrote dark/2015_02382 to the record.
Wrote dark/2015_02384 to the record.
Wrote dark/2015_02386 to the record.
Wrote dark/2015_02387 to the record.
Wrote dark/2015_02388 to the record.
Wrote dark/2015_02389 to the record.
Wrote dark/2015_02390 to the record.
Wrote dark/2015_02391 to the record.
Wrote dark/2015_02392 to the record.
Wrote dark/2015_02393 to the record.
Wrote dark/2015_02394 to the record.
Wrote dark/2015_02395 to the record.
Wrote dark/2015_02396 to the record.
Wrote dark/2015_02397 to the record.
Wrote dark/2015_02398 to the record.
Wrote dark/2015_02399 to the record.
Wrote dark/2015_02400 to the record.
Wrote dark/2015_02401 to the record.
Wrote dark/2015_02402 to the record.
Wrote dark/2015_02403 to the record.
Wrote dark/2015_02404 to the record.
W

Wrote dark/2015_02636 to the record.
Wrote dark/2015_02637 to the record.
Wrote dark/2015_02639 to the record.
Wrote dark/2015_02640 to the record.
Wrote dark/2015_02643 to the record.
Wrote dark/2015_02644 to the record.
Wrote dark/2015_02645 to the record.
Wrote dark/2015_02646 to the record.
Wrote dark/2015_02647 to the record.
Wrote dark/2015_02648 to the record.
Wrote dark/2015_02649 to the record.
Wrote dark/2015_02650 to the record.
Wrote dark/2015_02652 to the record.
Wrote dark/2015_02653 to the record.
Wrote dark/2015_02654 to the record.
Wrote dark/2015_02655 to the record.
Wrote dark/2015_02656 to the record.
Wrote dark/2015_02657 to the record.
Wrote dark/2015_02659 to the record.
Wrote dark/2015_02660 to the record.
Wrote dark/2015_02661 to the record.
Wrote dark/2015_02662 to the record.
Wrote dark/2015_02663 to the record.
Wrote dark/2015_02664 to the record.
Wrote dark/2015_02665 to the record.
Wrote dark/2015_02666 to the record.
Wrote dark/2015_02667 to the record.
W

Wrote dark/2015_02883 to the record.
Wrote dark/2015_02884 to the record.
Wrote dark/2015_02885 to the record.
Wrote dark/2015_02886 to the record.
Wrote dark/2015_02887 to the record.
Wrote dark/2015_02888 to the record.
Wrote dark/2015_02889 to the record.
Wrote dark/2015_02890 to the record.
Wrote dark/2015_02891 to the record.
Wrote dark/2015_02892 to the record.
Wrote dark/2015_02893 to the record.
Wrote dark/2015_02894 to the record.
Wrote dark/2015_02895 to the record.
Wrote dark/2015_02897 to the record.
Wrote dark/2015_02898 to the record.
Wrote dark/2015_02899 to the record.
Wrote dark/2015_02900 to the record.
Wrote dark/2015_02901 to the record.
Wrote dark/2015_02902 to the record.
Wrote dark/2015_02903 to the record.
Wrote dark/2015_02904 to the record.
Wrote dark/2015_02905 to the record.
Wrote dark/2015_02906 to the record.
Wrote dark/2015_02907 to the record.
Wrote dark/2015_02908 to the record.
Wrote dark/2015_02909 to the record.
Wrote dark/2015_02910 to the record.
W

...

Wrote kitti/006894 to the record.
Wrote kitti/006898 to the record.
Wrote kitti/006899 to the record.
Wrote kitti/006901 to the record.
Wrote kitti/006903 to the record.
Wrote kitti/006905 to the record.
Wrote kitti/006912 to the record.
Wrote kitti/006913 to the record.
Wrote kitti/006914 to the record.
Wrote kitti/006921 to the record.
Wrote kitti/006924 to the record.
Wrote kitti/006931 to the record.
Wrote kitti/006933 to the record.
Wrote kitti/006934 to the record.
Wrote kitti/006937 to the record.
Wrote kitti/006947 to the record.
Wrote kitti/006949 to the record.
Wrote kitti/006951 to the record.
Wrote kitti/006956 to the record.
Wrote kitti/006964 to the record.
Wrote kitti/006969 to the record.
Wrote kitti/006972 to the record.
Wrote kitti/006976 to the record.
Wrote kitti/006977 to the record.
Wrote kitti/006986 to the record.
Wrote kitti/006988 to the record.
Wrote kitti/006994 to the record.
Wrote kitti/006999 to the record.
Wrote kitti/007002 to the record.
Wrote kitti/00

Wrote raw/e39_aug_n_07-01-11-25 to the record.
Wrote raw/e39_aug_n_07-01-11-32 to the record.
Wrote raw/e39_aug_n_07-01-11-40 to the record.
Wrote raw/e39_aug_n_07-01-11-49 to the record.
Wrote raw/e39_aug_n_07-01-11-55 to the record.
Wrote raw/e39_aug_n_07-01-12-00 to the record.
Wrote raw/e39_aug_n_07-01-12-05 to the record.
Wrote raw/e39_aug_n_07-01-12-08 to the record.
Wrote raw/e39_aug_n_07-01-12-15 to the record.
Wrote raw/e39_aug_n_07-01-12-21 to the record.
Wrote raw/e39_aug_n_07-01-12-26 to the record.
Wrote raw/e39_aug_n_07-01-12-31 to the record.
Wrote raw/e39_aug_n_07-01-12-39 to the record.
Wrote raw/e39_aug_n_07-01-12-42 to the record.
Wrote raw/e39_aug_n_07-01-12-47 to the record.
Wrote raw/e39_aug_n_07-01-13-01 to the record.
Wrote raw/e39_aug_n_07-01-13-04 to the record.
Wrote raw/e39_aug_n_07-01-13-07 to the record.
Wrote raw/e39_aug_n_07-01-13-15 to the record.
Wrote raw/e39_aug_n_07-01-13-20 to the record.
Wrote raw/e39_aug_n_07-01-13-26 to the record.
Wrote raw/e39

Wrote raw/e39_aug_n_08-01-16-23 to the record.
Wrote raw/e39_aug_n_08-01-16-27 to the record.
Wrote raw/e39_aug_n_08-01-16-32 to the record.
Wrote raw/e39_aug_n_08-01-16-40 to the record.
Wrote raw/e39_aug_n_08-01-16-47 to the record.
Wrote raw/e39_aug_n_08-01-16-49 to the record.
Wrote raw/e39_aug_n_08-01-16-54 to the record.
Wrote raw/e39_aug_n_08-01-17-00 to the record.
Wrote raw/e39_aug_n_08-01-17-17 to the record.
Wrote raw/e39_aug_n_08-01-17-24 to the record.
Wrote raw/e39_aug_n_08-01-17-25 to the record.
Wrote raw/e39_aug_n_08-01-17-31 to the record.
Wrote raw/e39_aug_n_08-01-17-41 to the record.
Wrote raw/e39_aug_n_08-01-17-47 to the record.
Wrote raw/e39_aug_n_08-01-17-57 to the record.
Wrote raw/e39_aug_n_08-01-18-02 to the record.
Wrote raw/e39_aug_n_08-01-18-08 to the record.
Wrote raw/e39_aug_n_08-01-18-13 to the record.
Wrote raw/e39_aug_n_08-01-18-23 to the record.
Wrote raw/e39_aug_n_08-01-18-29 to the record.
Wrote raw/e39_aug_n_08-01-18-34 to the record.
Wrote raw/e39

Wrote raw/e39_aug_s_07-01-16-49 to the record.
Wrote raw/e39_aug_s_07-01-16-54 to the record.
Wrote raw/e39_aug_s_07-01-17-00 to the record.
Wrote raw/e39_aug_s_07-01-17-05 to the record.
Wrote raw/e39_aug_s_07-01-17-08 to the record.
Wrote raw/e39_aug_s_07-01-17-12 to the record.
Wrote raw/e39_aug_s_07-01-17-19 to the record.
Wrote raw/e39_aug_s_07-01-17-23 to the record.
Wrote raw/e39_aug_s_07-01-17-24 to the record.
Wrote raw/e39_aug_s_07-01-17-33 to the record.
Wrote raw/e39_aug_s_07-01-17-39 to the record.
Wrote raw/e39_aug_s_07-01-17-40 to the record.
Wrote raw/e39_aug_s_07-01-17-47 to the record.
Wrote raw/e39_aug_s_07-01-17-50 to the record.
Wrote raw/e39_aug_s_07-01-17-55 to the record.
Wrote raw/e39_aug_s_07-01-17-58 to the record.
Wrote raw/e39_aug_s_07-01-18-04 to the record.
Wrote raw/e39_aug_s_07-01-18-09 to the record.
Wrote raw/e39_aug_s_07-01-18-16 to the record.
Wrote raw/e39_aug_s_07-01-18-22 to the record.
Wrote raw/e39_aug_s_07-01-18-27 to the record.
Wrote raw/e39

Wrote raw/e39_aug_s_19-01-15-20 to the record.
Wrote raw/e39_vatland_09-01-12-33 to the record.
Wrote raw/e39_vatland_09-01-18-40 to the record.
Wrote raw/e39_vatland_10-01-16-07 to the record.
Wrote raw/e39_vatland_10-01-16-40 to the record.
Wrote raw/e39_vatland_10-01-17-19 to the record.
Wrote raw/e39_vatland_10-01-18-40 to the record.
Wrote raw/e6_nostvet_11-01-14-57 to the record.
Wrote raw/e6_nostvet_11-01-15-07 to the record.
Wrote raw/e6_nostvet_11-01-15-17 to the record.
Wrote raw/e6_nostvet_11-01-15-22 to the record.
Wrote raw/e6_nostvet_11-01-15-43 to the record.
Wrote raw/e6_nostvet_11-01-15-55 to the record.
Wrote raw/e6_nostvet_11-01-16-12 to the record.
Wrote raw/e6_nostvet_11-01-16-16 to the record.
Wrote raw/e6_nostvet_11-01-16-21 to the record.
Wrote raw/e6_nostvet_11-01-16-32 to the record.
Wrote raw/e6_nostvet_11-01-16-47 to the record.
Wrote raw/e6_nostvet_11-01-16-56 to the record.
Wrote raw/e6_nostvet_11-01-17-01 to the record.
Wrote raw/e6_nostvet_11-01-17-06 to

In [9]:
convertFolder('test', '../images/test', '../annotations')

Creating TF-record: ../annotations/test.record
Wrote dark/2015_01887 to the record.
Wrote dark/2015_01888 to the record.
Wrote dark/2015_01895 to the record.
Wrote dark/2015_01901 to the record.
Wrote dark/2015_01902 to the record.
Wrote dark/2015_01904 to the record.
Wrote dark/2015_01924 to the record.
Wrote dark/2015_01926 to the record.
Wrote dark/2015_01927 to the record.
Wrote dark/2015_01929 to the record.
Wrote dark/2015_01937 to the record.
Wrote dark/2015_01967 to the record.
Wrote dark/2015_01972 to the record.
Wrote dark/2015_01989 to the record.
Wrote dark/2015_01992 to the record.
Wrote dark/2015_02011 to the record.
Wrote dark/2015_02029 to the record.
Wrote dark/2015_02030 to the record.
Wrote dark/2015_02033 to the record.
Wrote dark/2015_02053 to the record.
Wrote dark/2015_02057 to the record.
Wrote dark/2015_02079 to the record.
Wrote dark/2015_02080 to the record.
Wrote dark/2015_02100 to the record.
Wrote dark/2015_02107 to the record.
Wrote dark/2015_02116 to the

Wrote dark/2015_06836 to the record.
Wrote dark/2015_06844 to the record.
Wrote dark/2015_06848 to the record.
Wrote dark/2015_06850 to the record.
Wrote kitti/000026 to the record.
Wrote kitti/000155 to the record.
Wrote kitti/000206 to the record.
Wrote kitti/000215 to the record.
Wrote kitti/000235 to the record.
Wrote kitti/000253 to the record.
Wrote kitti/000306 to the record.
Wrote kitti/000308 to the record.
Wrote kitti/000418 to the record.
Wrote kitti/000479 to the record.
Wrote kitti/000580 to the record.
Wrote kitti/000596 to the record.
Wrote kitti/000642 to the record.
Wrote kitti/000721 to the record.
Wrote kitti/000723 to the record.
Wrote kitti/000743 to the record.
Wrote kitti/000783 to the record.
Wrote kitti/000804 to the record.
Wrote kitti/000816 to the record.
Wrote kitti/000908 to the record.
Wrote kitti/000961 to the record.
Wrote kitti/000984 to the record.
Wrote kitti/001036 to the record.
Wrote kitti/001049 to the record.
Wrote kitti/001067 to the record.
Wr

Wrote kitti/007017 to the record.
Wrote kitti/007023 to the record.
Wrote kitti/007045 to the record.
Wrote kitti/007073 to the record.
Wrote kitti/007082 to the record.
Wrote kitti/007103 to the record.
Wrote kitti/007111 to the record.
Wrote kitti/007128 to the record.
Wrote kitti/007143 to the record.
Wrote kitti/007176 to the record.
Wrote kitti/007197 to the record.
Wrote kitti/007218 to the record.
Wrote kitti/007224 to the record.
Wrote kitti/007265 to the record.
Wrote kitti/007318 to the record.
Wrote kitti/007320 to the record.
Wrote kitti/007331 to the record.
Wrote kitti/007375 to the record.
Wrote kitti/007412 to the record.
Wrote kitti/007413 to the record.
Wrote kitti/007420 to the record.
Wrote kitti/007432 to the record.
Wrote kitti/007438 to the record.
Wrote kitti/007449 to the record.
Wrote raw/e39_aug_n_06-01-17-39 to the record.
Wrote raw/e39_aug_n_06-01-18-54 to the record.
Wrote raw/e39_aug_n_06-01-20-01 to the record.
Wrote raw/e39_aug_n_06-01-20-06 to the reco

In [6]:
#for record in tf.data.TFRecordDataset("../annotations/test.record"):
#    print(tf.train.SequenceExample.FromString(record.numpy()))